# Recommendation Model

## Preview
___

## Index
___

## Set up

### Import Module
___

In [2]:
import os
import sys
import pickle
import random

import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

import tensorflow as tf

In [3]:
random_seed = 1234

random.seed(random_seed)
np.random.seed(random_seed)
tf.random.set_seed(random_seed)

In [4]:
%matplotlib inline

plt.style.use('seaborn')
mpl.rcParams['figure.figsize'] = (10, 10)
mpl.rcParams['xtick.labelbottom'] = False
mpl.rcParams['ytick.labelleft'] = False
mpl.rcParams['axes.grid'] = False
mpl.rcParams.update({'figure.autolayout': True})

warnings.filterwarnings('ignore')

## Preface
___

### Recommend system

_What is recommendation system?_
> 나와 성향이 비슷한 사람들이 좋아하는 것을 나에게 추천한다.

There are __two parts__ of recommendation algorithm.

1. Collarative Filtering
2. Content-based Filtering

### <font color='red'>Collarative Filtering</font>
_협업 필터링_
___

<font size=4><b>기존 사용자의 행동 패턴을 분석하여 비슷한 이용자에게 기존 사용자의 패턴대로 추천하는 것.  
대표적인 예로 '이 상품을 구매한 사용자가 같이 본 상품'을 들 수 있다.  </b></font>
<span style="line-height:0.5em">
</br>  
대표 알고리즘 
 - Matrix Factorization  
 - k-nearest neighbor algorithm; kNN

<p>

단점 
 - Cold Start : 신곡과 같은 분석되지 않은 정보로는 추천할 수 없다. 
 - Long tail : 비대칭적으로 쏠린 정보에 의해 사용자들에게 관심이 없는 자료는 분석하기 어려워 충분한 정보를 제공할 수 없다.


### <font color='red'>Content-based Filtering</font>
_콘텐츠 기반 필터링_
___

<font size=4> __사용자가 아닌 컨텐츠 자체를 분석하는 것으로 비슷한 결의 콘텐츠를 추천하는 것__  
item profile(항목 분석)과 user profile(사용자의 선호도)의 유사성을 측정한 뒤 사용자가 좋아할 만한 컨텐츠를 제공한다.  </font>  
<span style="line-height:0.5em">
</br>
대표 알고리즘 
 - Clustrering analysis(군집 분석)  
 - Artificial neural network
 - tf-dif(term frequency-inverse document frequency)

### Social Issue
___

___Filter Bubble___

추천 시스템이 발전함에 따라서 사용자의 기존 성향대로 편향되게 정보에 노출되는 현상  
일반 대중들은 경각심을 가질 필요가 있다고 생각한다.  


### Rating Matrix
_평가 행렬_  
_row : user_  
_column : content_  

#### Matrix Factorization
_MF_



# 진행

# 참고
___

http://dl.moazine.com/viewer/index.asp?libraryid=9MtJb2T3nzH3yk7c212iu821DPFsY2ECOso0&a_i=8wwot4affaz3r6syx11VOYm2&keyword=&s_i=9#page/4